In [2]:
import bilby
from gwpy.timeseries import TimeSeries

/opt/miniconda3/envs/igwn-py310/lib/python3.10/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


In [3]:
logger = bilby.core.utils.logger
outdir = "outdir"
label = "GW170814"

In [4]:
from gwosc.datasets import event_gps
gps = event_gps('GW170814')
print("GW170814 GPS: ", gps)

GW170814 GPS:  1186741861.5


In [5]:
#gps is our trigger_time

In [6]:
detectors = ["H1" , "L1"]
max_frequency = 512
min_frequency = 20
roll_off = 0.4
duration = 4
post_trigger_duration = 2
end_time = gps + post_trigger_duration
start_time = end_time - duration

In [7]:
psd_duration = 32 * duration
psd_start_time = start_time - psd_duration
psd_end_time = start_time

In [8]:
#now we get analysis data and create ifo_list

In [9]:
ifo_list = bilby.gw.detector.InterferometerList([])
for det in detectors:
    logger.info("Downloading analysis data for ifo {}".format(det))
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    data = TimeSeries.fetch_open_data(det, start_time , end_time)
    ifo.strain_data.set_from_gwpy_timeseries(data)
    
    logger.info("Downloadng psd data for ifo {}".format(det))
    psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time)
    psd_alpha = 2*roll_off / duration
    psd = psd_data.psd(
         fftlength = duration, overlap = 0, window =("tukey", psd_alpha), method = "median"
    )
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array = psd.frequencies.value, psd_array = psd.value
    )
    ifo.maximum_frequency = max_frequency
    ifo.minimum_frequenxy = min_frequency
    ifo_list.append(ifo)

logger.info("Saving data plots to {}".format(outdir))
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)
ifo_list.plot_data(outdir=outdir, label=label)

00:52 bilby INFO    : Downloading analysis data for ifo H1
00:53 bilby INFO    : Downloadng psd data for ifo H1
00:54 bilby INFO    : Downloading analysis data for ifo L1
00:54 bilby INFO    : Downloadng psd data for ifo L1
00:57 bilby INFO    : Saving data plots to outdir


In [10]:
#now we define prior

In [19]:
priors = bilby.gw.prior.BBHPriorDict(filename = "GW170814.prior")
print(priors)

{'mass_1': Constraint(minimum=20, maximum=40, name='mass_1', latex_label='$m_1$', unit='$M_{\\odot}$'), 'mass_2': Constraint(minimum=20, maximum=40, name='mass_2', latex_label='$m_2$', unit='$M_{\\odot}$'), 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False), 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=25, maximum=100, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=None), 'luminosity_distance': PowerLaw(alpha=2, minimum=300, maximum=3000, name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None), 'dec': Cosine(minimum=-1.5707963267948966, maximum=1.5707963267948966, name='dec', latex_label='$\\mathrm{DEC}$', unit=None, boundary=None), 'ra': Uniform(minimum=0, maximum=6.283185307179586, name='ra', latex_label='$\\mathrm{RA}$', unit=None, boundary='periodic'), 'theta_jn': Sine(minimum=0, maximum=3.141592653589793, 

In [20]:
priors["geocent_time"] = bilby.core.prior.Uniform(
    minimum=gps - 0.1,
    maximum=gps + 0.1,
    name = "geocent_time"
)

In [21]:
#now we define a waveform generator which creates a frequency domain strain, 
#we will use 'lal binary_black_hole model' as our source model. Other parameters - the waveform approximant and 
#reference frequency and a parameter conversion
# which allows us to sample in chirp mass and ratio rather than component mass


In [22]:
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={
        "waveform_approximant": "IMRPhenomPv2",
        "reference_frequency": 50,
    },
)

01:04 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [23]:
#Now define the likelihood. standard likelihood function, passing it the data and the waveform generator.
#phase_marginalization is formally invalid with a precessing waveform such as IMRPhenomPv2

In [27]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator,
    priors=priors,
    time_marginalization=True,
    phase_marginalization=False,
    distance_marginalization=True,
)


01:04 bilby INFO    : Loaded distance marginalisation lookup table does not match for distance_array.
01:04 bilby INFO    : Building lookup table for distance marginalisation.


  0%|          | 0/400 [00:00<?, ?it/s]

In [28]:
#Now we run the sampler

In [ ]:
result = bilby.run_sampler(
    likelihood,
    priors,
    sampler="dynesty",
    outdir=outdir,
    label=label,
    nlive=1000,
    check_point_delta_t=600,
    check_point_plot=True,
    npool=1,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
)
result.plot_corner()


01:05 bilby INFO    : Running for label 'GW170814', output will be saved to 'outdir'
01:05 bilby INFO    : Using lal version 7.6.1
01:05 bilby INFO    : Using lal git version Branch: None;Tag: lal-v7.6.1;Id: 31af23159ed7c6557180f58ad3f39a06e5a08731;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
01:05 bilby INFO    : Using lalsimulation version 6.1.0
01:05 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsimulation-v6.1.0;Id: 8041734408377ca999821f7e372e2a02e9226e6b;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
01:05 bilby INFO    : Analysis priors:
01:05 bilby INFO    : mass_ratio=bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False)
01:05 bilby INFO    : chirp_mass=bilby.gw.prior.UniformInComponentsChirpMass(minimum=25, maximum=100, name='chirp_mass', latex_label='$

1it [00:00, ?it/s]

/opt/miniconda3/envs/igwn-py310/lib/python3.10/site-packages/dynesty/sampler.py:761: UserWarning: The sampling was stopped short due to maxiter/maxcall limit the delta(log(z)) criterion is not achieved; posterior may be poorly sampled
  warnings.warn('The sampling was stopped short due to'
